Introducing manual compression of image captions on stale (offline) data

In [1]:
%pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
from transformers import VisionEncoderDecoderModel, ViTFeatureExtractor, AutoTokenizer, BitsAndBytesConfig, AutoProcessor, LlavaForConditionalGeneration
from transformers import AdamW
from datasets import load_dataset
import torch
from collections import Counter
import fiftyone
import torch.nn as nn
from tqdm.auto import tqdm
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as T
import torch.nn.functional as F
import numpy as np
import os
import time
import glob

# uncommon features  - events of interest
# loss less compression -  sudden more bits indicates anomaly can be flagged, alerts when anomaly detected - may shift to lossy video streaming
# lossy compression of noisy data varying distortion rate - accuracy is increasing
# video to video lossy reconstruction possibility
# image frame to image frame on a need basis - human satisfaction metric, GPT based comparison, RLHF based comparison

/Users/sagnik/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Check if GPU is available and set the device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
# Load the pre-trained model and its components
model = VisionEncoderDecoderModel.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
feature_extractor = ViTFeatureExtractor.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
tokenizer = AutoTokenizer.from_pretrained("nlpconnect/vit-gpt2-image-captioning")

# # Loading the above for LlavVA
# model_llava = LlavaForConditionalGeneration.from_pretrained("llava-hf/llava-1.5-7b-hf")
# processor_llava = AutoProcessor.from_pretrained("llava-hf/llava-1.5-7b-hf")

/Users/sagnik/Library/Python/3.9/lib/python/site-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [5]:
# Load a dataset (for example, a subset of the COCO dataset)
# TODO: Potential datasets with repetitive nature that can be used: MS COCO, Flickr30k, Visual Genome, SBU Captions - get correlated datasets from Nikil

# load small part of the coco dataset from all the .jpg images in datasets/mscoco/test2015
dataset = load_dataset("datasets/mscoco/test2015/", split="test[:50]")

Resolving data files: 100%|██████████| 81434/81434 [00:00<00:00, 312128.37it/s]


In [6]:
# TODO: use different values of max_length and try out results

def generate_caption_with_logits(image, max_length=15):
    # define device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # Prepare the inputs
    inputs = feature_extractor(images=image, return_tensors="pt").to(device)
    pixel_values = inputs.pixel_values
    
    model.to(device)
    model.eval()
    
    with torch.no_grad():
        # Perform a forward pass to get the logits
        encoder_outputs = model.encoder(pixel_values=pixel_values)
        encoder_hidden_states = encoder_outputs.last_hidden_state
        
        # Prepare decoder input_ids. Typically, you start with the start-of-sentence token
        decoder_input_ids = torch.tensor([tokenizer.bos_token_id]).unsqueeze(0).to(encoder_hidden_states.device)
        decoder_attention_mask = torch.ones_like(decoder_input_ids)
        
        # Initialize an empty tensor for logits (for simplicity, accumulating logits for each step)
        logits_list = []
        
        for i in range(max_length):
            decoder_outputs = model.decoder(input_ids=decoder_input_ids,
                                            attention_mask=decoder_attention_mask,
                                            encoder_hidden_states=encoder_hidden_states)
            logits = decoder_outputs.logits[:, -1, :]  # Get the logits for the last token generated
            logits_list.append(logits)
            
            predicted_id = torch.argmax(logits, dim=-1).unsqueeze(-1)
            # Check if EOS token is generated
            if predicted_id[0, 0] == tokenizer.eos_token_id:
                print ("EOS has been generated")
                # break # since model.generate() does this automatically
            
            # Append predicted token ID to decoder_input_ids for generating next token
            decoder_input_ids = torch.cat([decoder_input_ids, predicted_id], dim=-1)
            decoder_attention_mask = torch.cat([decoder_attention_mask, torch.ones_like(predicted_id).to(device)], dim=-1)
            
        # Concatenate logits from each step to get the final logits tensor
        # make all elements of logits_list 3D by adding a dimension in the middle
        logits_list = [logits.unsqueeze(1) for logits in logits_list]
        logits = torch.cat(logits_list, dim=1)
        # add logic to repeat the remaining number of (127-i) tokens with EOS token logits (simply repeat the last token logits) to make it length 128

        # Decode the generated token IDs to get the caption
        predicted_ids = torch.argmax(logits, dim=-1)
        caption = tokenizer.decode(predicted_ids[0], skip_special_tokens=True)
        
    return logits, predicted_ids, caption

# Example usage
# image: A PIL image or a tensor representing your input image
# logits, predicted_ids, caption = generate_caption_with_logits(image, model, feature_extractor, tokenizer)


In [7]:
# Iterate over the dataset and generate captions
generated_captions = []
generated_logits = []
generated_predicted_ids = []

for data in dataset:
    image = data['image']
    logits, predicted_ids, caption = generate_caption_with_logits(image)
    generated_captions.append(caption)
    generated_logits.append(logits)
    generated_predicted_ids.append(predicted_ids)

# concatenate generated logits along first dimension to make 3D tensor
generated_logits = torch.cat(generated_logits, dim=0)
print (generated_logits.shape)

# concatenate generated predicted_ids along first dimension to make 2D tensor
generated_predicted_ids = torch.cat(generated_predicted_ids, dim=0)

# form new tensor of size unique tokens * vocab size : each row is the means of logits of all tokens with that id\n",
# find unique elements in generated_predicted_ids, result should be a 1D tensor\n",
unique_tokens = torch.unique(generated_predicted_ids)
# store torch tensor of the number of times each unique token appears in generated_predicted_ids -  remember generated_predicted_ids is a 2D tensor
unique_token_counts = torch.zeros(unique_tokens.shape[0]).to(device)
for i, token in enumerate(unique_tokens):
    unique_token_counts[i] = torch.sum(generated_predicted_ids == token)
unique_token_probs = unique_token_counts / torch.sum(unique_token_counts)
print (unique_token_counts.shape)
# find the mean of logits for each unique token
mean_logits = torch.zeros(unique_tokens.shape[0], generated_logits.shape[2]).to(device)
for i, token in enumerate(unique_tokens):
    mean_logits[i] = torch.mean(generated_logits[generated_predicted_ids == token], dim=0)
print (mean_logits.shape)

EOS has been generated
EOS has been generated
EOS has been generated
EOS has been generated
EOS has been generated
EOS has been generated
EOS has been generated
EOS has been generated
EOS has been generated
EOS has been generated
EOS has been generated
EOS has been generated
EOS has been generated
EOS has been generated
EOS has been generated
EOS has been generated
EOS has been generated
EOS has been generated
EOS has been generated
EOS has been generated
EOS has been generated
EOS has been generated
EOS has been generated
EOS has been generated
EOS has been generated
EOS has been generated
EOS has been generated
EOS has been generated
EOS has been generated
EOS has been generated
EOS has been generated
EOS has been generated
EOS has been generated
EOS has been generated
EOS has been generated
EOS has been generated
EOS has been generated
EOS has been generated
EOS has been generated
EOS has been generated
EOS has been generated
EOS has been generated
EOS has been generated
EOS has bee

In [8]:
generated_captions

['a green truck parked next to a curb a green truck parked next',
 'a man is walking down the street with a skateboard a man',
 'a baseball player swinging a bat at a ball a baseball player swinging',
 'a cow is standing in a field of grass a cow is standing',
 'a black dog sitting in the back of a truck a black dog',
 'a man wearing a bow tie and glasses a man wearing a bow',
 'a dining room table with a large bowl of food a large kitchen',
 'a man standing next to a wall with a bunch of guitars a',
 'a man is playing tennis on a clay court a man is playing',
 'a man and a woman playing a game of frisbee a',
 'a woman and a man are drinking wine people are sitting at a',
 'people are standing around a table people are drinking wine people',
 'a zebra standing in a fenced in area a zebra',
 'two dogs are looking at each other in a room a dog is',
 'a horse grazing in a field with a tree a horse grazing in',
 'a bird perched on top of a bird feeder a bird sitting',
 'a train on a track 

In [9]:
def update_encoding_dict(captions, encoding_dict):
    for caption in captions:
        words = caption.split() # splitting the caption into words - pretty bad strategy since we are currently splitting into tokens
        encoding_dict.update(words) # purpose of update is to add the words to the dictionary if they don't exist
    return encoding_dict

In [10]:
encoding_dict = Counter() # Counter is a subclass of dictionary for counting hashable objects
threshold = 0 # threshold for word frequency # TODO: find a good threshold

update_encoding_dict(generated_captions, encoding_dict)

print (encoding_dict)

# Optionally, create a more compressed form based on frequency
compressed_dict = {word: idx for idx, (word, freq) in enumerate(encoding_dict.items()) if freq > threshold}

# Create the dictionary of entropy values from encoding_dict
entropy_dict = {word: -np.log(encoding_dict[word] / sum(encoding_dict.values())) 
                for word in encoding_dict}

print (entropy_dict)
# print 1/elem for elem in encoding_dict.values()
reciprocal_dict = {word: 1/(encoding_dict[word]+1) for word in encoding_dict}
print (reciprocal_dict)

Counter({'a': 167, 'man': 24, 'on': 22, 'of': 21, 'with': 20, 'is': 14, 'in': 13, 'and': 12, 'standing': 10, 'sitting': 10, 'food': 9, 'are': 8, 'people': 8, 'truck': 7, 'large': 7, 'at': 6, 'woman': 6, 'plate': 6, 'parked': 5, 'playing': 5, 'tennis': 5, 'next': 4, 'to': 4, 'the': 4, 'dog': 4, 'room': 4, 'bear': 4, 'clock': 4, 'walking': 3, 'field': 3, 'black': 3, 'wearing': 3, 'tie': 3, 'table': 3, 'court': 3, 'grazing': 3, 'bird': 3, 'top': 3, 'person': 3, 'teddy': 3, 'motorcycle': 3, 'riding': 3, 'tower': 3, 'green': 2, 'down': 2, 'street': 2, 'skateboard': 2, 'baseball': 2, 'player': 2, 'swinging': 2, 'cow': 2, 'bow': 2, 'bowl': 2, 'kitchen': 2, 'drinking': 2, 'wine': 2, 'zebra': 2, 'looking': 2, 'horse': 2, 'tree': 2, 'train': 2, 'track': 2, 'fence': 2, 'vegetables': 2, 'an': 2, 'elephant': 2, 'watching': 2, 'cooking': 2, 'meat,': 2, 'suit': 2, 'holding': 2, 'couch': 2, 'side': 2, 'through': 2, 'bed': 2, 'herd': 2, 'sheep': 2, 'remote': 2, 'control': 2, 'jetliner': 2, 'jumping': 2

In [11]:
class CaptionDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, captions):
        self.encodings = encodings
        self.captions = captions

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.captions[idx])
        return item

    def __len__(self):
        return len(self.captions)

In [12]:
# Assuming `dataset` is your dataset containing images and captions
images = [data['image'] for data in dataset]
caption_ids = generated_predicted_ids

# Process images and captions
inputs = feature_extractor(images=images, return_tensors="pt") 

# Create dataset and dataloader
train_dataset = CaptionDataset(inputs, caption_ids)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

In [13]:
class LoRALayer(nn.Module):
    def __init__(self, original_weight, rank):
        super(LoRALayer, self).__init__()
        self.original_weight = original_weight
        self.rank = rank
        self.device = original_weight.device
        self.U = nn.Parameter(torch.Tensor(self.original_weight.size(0), self.rank)).to(self.device)
        self.V = nn.Parameter(torch.Tensor(self.rank, self.original_weight.size(1))).to(self.device)
        nn.init.xavier_uniform_(self.U)
        nn.init.xavier_uniform_(self.V)

    def forward(self):
        return self.original_weight + self.U @ self.V

In [14]:
# Modify the first attention layer of the encoder
# TODO: Try modifying other layers as well and check the results
lora_layers = []

with torch.no_grad():
    original_weight = model.encoder.encoder.layer[0].attention.output.dense.weight
    lora_layer = LoRALayer(original_weight, rank=10).to(device).forward()  # Choose an appropriate rank
    # assign the new layer to the model
    model.encoder.encoder.layer[0].attention.output.dense.weight.copy_(lora_layer)
    # add the layer of the model to the list of LoRA layers
    lora_layers.append(model.encoder.encoder.layer[0].attention.output.dense)

In [15]:
def is_lora_param(param, lora_layer):
    # check if the parameter is part of the LoRA layer
    print (lora_layer.parameters())
    print ("nuj")
    print (param)
    return param in lora_layer.parameters()


In [16]:
# add two extra dimensions to generated_logits
generated_probs = F.softmax(mean_logits, dim=-1)
generated_probs_expanded = generated_probs.unsqueeze(0).unsqueeze(0).to(device)

In [17]:
def calculate_entropy_elbo_difference (prob_differences, unique_token_counts, D):
    sigma = 0.01
    # reduce prob_differences to 4D from 5D by taking norm square along the last dimension
    prob_differences = prob_differences.to(device)
    prob_differences = torch.norm(prob_differences, dim=-1)
    print (prob_differences.shape)
    # do elementwise for prob_differences: suqare
    prob_differences = prob_differences**2
    # multiply i,j,k th element of prob_differences with k th element of unique_token_probs
    prob_differences = prob_differences * unique_token_counts
    # take sum of all elements of prob_differences, hence scalar, then divide by 2*sigma^2*D
    return torch.sum(prob_differences) / (2*sigma**2*D)

In [18]:
def calculate_entropy_elbo_cross_entropy (prob_differences, D):
    pass

In [19]:
def calculate_entropy (prob_differences, D):
    pass

In [30]:
def custom_loss(outputs, batch, encoding_dict, lora_layers, lambda_val=0.05, lora_lambda_val = 0.01):
    # Sending all to GPU
    outputs.logits = outputs.logits.to(device)
    outputs.loss = outputs.loss.to(device)
    
    # Standard captioning loss
    standard_loss = outputs.loss

    # Additional compression loss
    compression_loss = 0
    # add two dimensions to output probs at 2 and 3
    outputs_probs = F.softmax(outputs.logits, dim=-1)
    outputs_probs_expanded = outputs_probs.squeeze(1).unsqueeze(2).to(device)
    # print shapes of generated_probs_expanded and outputs_probs_expanded
    print ("generated_probs_expanded.shape = ", generated_probs_expanded.shape, outputs_probs_expanded.shape)
    prob_differences = generated_probs_expanded - outputs_probs_expanded
    print ("prob_differences.shape = ", outputs_probs.shape, generated_probs_expanded.shape, outputs_probs_expanded.shape, prob_differences.shape)
    # calculate the compression loss
    # find number of elements in generated_predicted_logits
    D = generated_probs.numel()
    compression_loss = lambda_val* calculate_entropy_elbo_difference (prob_differences, unique_token_counts, D)
    

    # Optionally, add a term for LoRA regularization if needed
    lora_regularization = 0
    # for param in model.parameters():
    #     for lora_layer in lora_layers:
    #         if is_lora_param(param, lora_layer):
    #             lora_regularization += torch.norm(param)
    print (standard_loss, compression_loss)

    return standard_loss + compression_loss + lora_lambda_val * lora_regularization

In [31]:
# Fine tuning using custom loss

#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.train()

lr = 1e-4
num_epochs = 2

optimizer = AdamW([param for param in model.parameters() if param.requires_grad], lr=lr)

for epoch in range(num_epochs):
    loop = tqdm(train_loader, leave=True)
    for batch in loop:
        # Move batch to device
        batch = {k: v.to(device) for k, v in batch.items()}

        # Forward pass
        outputs = model(**batch)
        loss = custom_loss(outputs, batch, encoding_dict, lora_layers)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Update progress bar
        loop.set_description(f"Epoch {epoch}")
        loop.set_postfix(loss=loss.item())

/Users/sagnik/Library/Python/3.9/lib/python/site-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  0%|          | 0/7 [00:00<?, ?it/s]/var/folders/wd/7s_rgclx5rlc79rrjnspznh00000gn/T/ipykernel_23228/3431795457.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
/var/folders/wd/7s_rgclx5rlc79rrjnspznh00000gn/T/ipykernel_23228/3431795457.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  

generated_probs_expanded.shape =  torch.Size([1, 1, 182, 50257]) torch.Size([8, 15, 1, 50257])
prob_differences.shape =  torch.Size([8, 15, 50257]) torch.Size([1, 1, 182, 50257]) torch.Size([8, 15, 1, 50257]) torch.Size([8, 15, 182, 50257])
torch.Size([8, 15, 182])
tensor(5.8525, grad_fn=<NllLossBackward0>) tensor(2.6791, grad_fn=<MulBackward0>)


Epoch 0:  14%|█▍        | 1/7 [00:15<01:35, 16.00s/it, loss=8.53]

generated_probs_expanded.shape =  torch.Size([1, 1, 182, 50257]) torch.Size([8, 15, 1, 50257])
prob_differences.shape =  torch.Size([8, 15, 50257]) torch.Size([1, 1, 182, 50257]) torch.Size([8, 15, 1, 50257]) torch.Size([8, 15, 182, 50257])
torch.Size([8, 15, 182])
tensor(5.6312, grad_fn=<NllLossBackward0>) tensor(2.6961, grad_fn=<MulBackward0>)


Epoch 0:  29%|██▊       | 2/7 [00:30<01:14, 14.93s/it, loss=8.33]

...

Epoch 1:  71%|███████▏  | 5/7 [01:09<00:26, 13.04s/it, loss=6.16]

generated_probs_expanded.shape =  torch.Size([1, 1, 182, 50257]) torch.Size([8, 15, 1, 50257])
prob_differences.shape =  torch.Size([8, 15, 50257]) torch.Size([1, 1, 182, 50257]) torch.Size([8, 15, 1, 50257]) torch.Size([8, 15, 182, 50257])
torch.Size([8, 15, 182])
tensor(3.2077, grad_fn=<NllLossBackward0>) tensor(2.9157, grad_fn=<MulBackward0>)


Epoch 1:  86%|████████▌ | 6/7 [01:26<00:14, 14.44s/it, loss=6.12]

generated_probs_expanded.shape =  torch.Size([1, 1, 182, 50257]) torch.Size([2, 15, 1, 50257])
prob_differences.shape =  torch.Size([2, 15, 50257]) torch.Size([1, 1, 182, 50257]) torch.Size([2, 15, 1, 50257]) torch.Size([2, 15, 182, 50257])
torch.Size([2, 15, 182])
tensor(2.9239, grad_fn=<NllLossBackward0>) tensor(0.7412, grad_fn=<MulBackward0>)


Epoch 1: 100%|██████████| 7/7 [01:27<00:00, 12.55s/it, loss=3.67]


In [32]:
# create directory to save the model if it doesn't exist
if not os.path.exists("models"):
    os.mkdir("models")
# save model checkpoint to models directory using current timestamp and date
torch.save(model.state_dict(), f"models/{time.strftime('%Y%m%d-%H%M%S')}.pth")


In [33]:
# load latest model checkpoint among all the saved models
latest_model = torch.load(max(glob.glob('models/*.pth'), key=os.path.getctime))
# load the model with the latest checkpoint
model.load_state_dict(latest_model)
model.to(device)

VisionEncoderDecoderModel(
  (encoder): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_featur

In [34]:
# Generate captions for the test dataset
generated_captions_custom_model = []
# Iterate over the dataset and generate captions
for data in dataset:
    image = data['image']
    generated_logits, generated_predicted_ids, caption = generate_caption_with_logits(image)
    generated_captions_custom_model.append(caption)


EOS has been generated
EOS has been generated
EOS has been generated
EOS has been generated
EOS has been generated
EOS has been generated
EOS has been generated


In [35]:
# Encode compressed dictionary word using manual huffman encoding

In [36]:
# Replace compressed_dict words occurring in the generated_captions_custom_model with their corresponding huffman encoding

In [27]:
# compare encoded generated_captions_custom_model + huffman encoding dictionary information with the original generated_captions to calculate compression ratio

In [38]:
# print generated_captions and generated_captions_custom_model elementwise to compare the results
for i in range(len(generated_captions)):
    print (generated_captions[i], "WAIT", generated_captions_custom_model[i])
# print the frequency of each word in generated_captions
print (encoding_dict)


a green truck parked next to a curb a green truck parked next WAIT a man is sitting on a man sitting on a man sitting on a man
a man is walking down the street with a skateboard a man WAIT a man is playing a man playing a man playing a man playing a man
a baseball player swinging a bat at a ball a baseball player swinging WAIT a man is playing a man playing a man playing a man playing a man
a cow is standing in a field of grass a cow is standing WAIT a man is playing a man playing a man playing a man playing a man
a black dog sitting in the back of a truck a black dog WAIT a man is sitting on a man sitting on a man sitting on a man
a man wearing a bow tie and glasses a man wearing a bow WAIT a man is a man is a man is a man is a man is
a dining room table with a large bowl of food a large kitchen WAIT a man is sitting on a man sitting on a man sitting on a man
a man standing next to a wall with a bunch of guitars a WAIT a man is a man is a man is a man is a man is
a man is playing tenn